In [ ]:
from scipy.stats import norm
import numpy as np

def sensor_model(z, x, l, z_r_variance, z_theta_variance):
    """
    x - 2-tuple [x_r,y_r] - robot location
    l - 2-tuple [x_l,y_l] - landmark location
    z - 2-tuple [z_r, z_theta] - sensor reading to landmark location

    Theta orientation is in radians

    Implement the sensor model and return p(z|x,l)
    """

    # p(z|x,l) = p(z_r,z_theta|x,l)
    #
    # Since z_r and z_theta are independent, then:
    # p(z_r,z_theta|x,l) = p(z_r|x,l) * p(z_theta|x,l)
    #
    # p(z_r|x,l) = norm(z_r - z_r_hat, z_r_variance)
    # z_r_hat = distance(x,l)
    #
    # Similarly, p(z_theta|x,l) = norm(z_theta - z_theta_hat, z_theta_variance)
    # z_theta_hat = angle(x,l)

    z_r_hat = np.sqrt((l[1] - x[1])**2 + (l[0] - x[0])**2)
    p_z_r_given_x_l = norm.pdf(
        z[0] - z_r_hat, loc=0, scale=np.sqrt(z_r_variance)
    )

    # For theta, we need to make sure the angle between measurement
    # and that between x and l is within pi and -pi.
    z_r_theta = np.arctan2(l[1] - x[1], l[0] - x[0])
    z_diff = np.abs(z[1] - z_r_theta)
    z_diff = z_diff if z_diff <= np.pi else ((2.0 * np.pi) - z_diff)
    p_z_theta_given_x_l = norm.pdf(
        z_diff, loc=0, scale=np.sqrt(z_theta_variance)
    )

    p_of_z_given_x_l = p_z_r_given_x_l * p_z_theta_given_x_l
    return p_of_z_given_x_l

x = np.array([0,0])
l = np.array([10, 10])
z_01 = np.array([np.sqrt(10**2 + 10**2), np.pi * 0.25])
z_02 = np.array([np.sqrt(10**2 + 10**2) + 1, np.pi * 0.24])
z_r_variance = 1.5
z_theta_variance = 1.2
print(
    "The likelihood of measurement z_01 vs z_02:\n"
    f"{sensor_model(z_01, x, l, z_r_variance, z_theta_variance)} vs "
    f"{sensor_model(z_02, x, l, z_r_variance, z_theta_variance)}")